In [1]:
import os
import math

import matplotlib.pyplot as plt
from mpl_toolkits.axisartist.axislines import AxesZero
import matplotlib.gridspec as gridspec
from matplotlib import cm, transforms
import matplotlib.ticker as mtick
from mpl_axes_aligner import align
import numpy as np
np.seterr(divide='ignore')
from scipy.stats import poisson, norm, lognorm
from scipy import optimize as opti
import pandas as pd
from tqdm import tqdm
from scipy import special
from scipy.stats import norm
from scipy.stats import multivariate_normal
from scipy.signal import savgol_filter
import h5py
import torch

import wf_func as wff

np.random.seed(0)

In [2]:
Mu = 4
Tau = 20
Sigma = 5
file = '4.0-20-5'

In [3]:
with h5py.File('waveform/' + file + '.h5', 'r', libver='latest', swmr=True) as ipt:
    ent = ipt['Readout/Waveform'][:]
    tru = ipt['SimTriggerInfo/PEList'][:]
    gmu = ipt['SimTriggerInfo/PEList'].attrs['gmu']
    gsigma = ipt['SimTriggerInfo/PEList'].attrs['gsigma']
    t0truth = ipt['SimTruth/T'][:]

In [4]:
def normcombine(x, m, s, a):
    return a[0] * norm.pdf((x - m[0]) / s[0]) + a[1] * norm.pdf((x - m[1]) / s[1])

def normcombine2d(x, m, s, a, rho):
    return a[0, 0] * multivariate_normal.pdf(x, mean=[m[0, 0], m[1, 0]], cov=matrix(s[0, 0], s[1, 0], rho[0, 0])) + a[0, 1] * multivariate_normal.pdf(x, mean=[m[0, 0], m[1, 1]], cov=matrix(s[0, 0], s[1, 1], rho[0, 1])) + a[1, 0] * multivariate_normal.pdf(x, mean=[m[0, 1], m[1, 0]], cov=matrix(s[0, 1], s[1, 0], rho[1, 0])) + a[1, 1] * multivariate_normal.pdf(x, mean=[m[0, 1], m[1, 1]], cov=matrix(s[0, 1], s[1, 1], rho[1, 1]))

def matrix(sx, sy, rho):
    return np.array([[sx ** 2, rho * sx * sy], [rho * sx * sy, sy ** 2]])

def chargehist(t):
    c = norm.pdf(t, loc=gmu, scale=gsigma)
#     q1 = 150.8
#     sigma = 37.59
#     w = 2.433e-5
#     alpha = 0.01335
#     mu = 2.851e-5
#     c = np.exp(-mu)*(w*alpha*np.exp(-alpha*t))
#     c = c + mu*np.exp(-mu)*(
#         (1-w)/(sigma*np.sqrt(2*np.pi))*np.exp(-(t-q1)**2/(2*sigma**2))+
#         w*(alpha/2*np.exp(-alpha*(t-q1-alpha/2*sigma**2))*(1+special.erf(t-q1-alpha*sigma**2)/(np.sqrt(2)*sigma))))
    return c

In [5]:
Thres = wff.Thres
std = 1.
spe_pre = wff.read_model('spe.h5', 1)
p = spe_pre[0]['parameters']
window = wff.window
t_auto = np.arange(window).reshape(window, 1) - np.arange(window).reshape(1, window)
mnecpu = wff.spe((t_auto + np.abs(t_auto)) / 2, p[0], p[1], p[2])

In [6]:
fig = plt.figure(figsize=(8, 6))
t = np.arange(-4 * 5, 5 * 20, 0.1)
# gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.95, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
# ax = fig.add_subplot(gs[0, 0])
ax = fig.add_axes((.125, .12, .775, .77))
ax.plot(t, wff.convolve_exp_norm(t, 20, 0), label=r'$(20,0)$', color='g')
ax.plot(t, wff.convolve_exp_norm(t, 0, 5), label=r'$(0,5)$', color='r')
ax.plot(t, wff.convolve_exp_norm(t, 20, 5), label=r'$(20,5)$', color='b')
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.grid()
ax.set_xlim(xmin=-4 * int(5))
ax.set_ylabel(r'$\mathrm{PDF}$')
ax.legend(title=r'$(\tau_l, \sigma_l)/\si{ns}$', loc='upper right')
ax.set_ylim(0, ax.get_ylim()[1] * 1.05)
# ax.annotate(r'$t_{0}$', xy=(0, 0), xytext=(5, 0.01), arrowprops=dict(facecolor='k', shrink=0.1, width=0.1, headwidth=2))
fig.savefig('Note/figures/profile.pgf')
fig.savefig('Note/figures/profile.pdf')
plt.close()
ax.get_position()

Bbox([[0.125, 0.12], [0.9, 0.89]])

In [7]:
fig = plt.figure(figsize=(8, 4))
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.05, right=0.97, top=0.97, bottom=0.1, wspace=0.3, hspace=0.3)
ax = fig.add_subplot(gs[0, 0])

ax.spines['left'].set_position(('data', 0))
ax.spines['bottom'].set_position(('data', 0))
ax.plot(1, 0, '>k', transform=ax.get_yaxis_transform(), clip_on=False)
ax.plot(0, 1, '^k', transform=ax.get_xaxis_transform(), clip_on=False)

t = np.linspace(0, 6, 201)
ax.plot(t, lognorm.pdf(t, loc=0, s=0.3), color='darkorange')
ax.plot(t, lognorm.pdf(t, loc=3, s=0.3), color='darkblue')
ax.fill_between(t, 0, lognorm.pdf(t, loc=0, s=0.3), color='darkorange', alpha=0.5)
ax.fill_between(t, 0, lognorm.pdf(t, loc=3, s=0.3), color='darkblue', alpha=0.5)
ax.set_xlim(0, 6)
ax.set_ylim(bottom=1e-3)
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel(r'$\mathrm{Time}$')
ax.set_ylabel(r'$\mathrm{Voltage}$')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.annotate(text='', xy=(1.5, 1), xytext=(3.5, 1), arrowprops=dict(arrowstyle='<->'))
ax.text(x=2.2, y=1.1, s=r'$\sim D_w$')
ax.text(x=0.7, y=0.3, s=r'$\sim \mathrm{RSS}$')
ax.text(x=3.7, y=0.3, s=r'$\sim \mathrm{RSS}$')
fig.savefig('Note/figures/tab.pgf')
fig.savefig('Note/figures/tab.pdf')
fig.clf()
plt.close(fig)

In [8]:
i = 2
cid = ent[i]['ChannelID']
eid = ent[i]['TriggerNo']
truth = np.sort(tru[(tru['TriggerNo'] == eid) & (tru['PMTId'] == cid)], kind='stable', order=['TriggerNo', 'PMTId', 'HitPosInWindow'])
wave = ent[i]['Waveform'].astype(np.float) * spe_pre[cid]['epulse']
df = pd.DataFrame(truth)
df = df.rename(columns={'HitPosInWindow':'HitTime'})
charge = df['Charge'].copy()
hittime = df['HitTime'].copy()
df = df.astype({'Charge': 'float32'})
df = df.astype({'TriggerNo' : 'str', 'PMTId' : 'str', 'HitTime' : 'str', 'Charge': 'str'})
df['HitTime'] = ['{:.02f}'.format(s) for s in hittime]
df['Charge'] = ['{:.02f}'.format(s) for s in charge]
df

,TriggerNo,PMTId,HitTime,Charge
0,2,0,469.16,96.56
1,2,0,471.43,200.71
2,2,0,487.39,220.02
3,2,0,499.50,165.30
4,2,0,527.69,188.54


In [9]:
ind = np.argwhere(wave > spe_pre[cid]['std'] * 5).flatten()
xmin = ((ind.min() - spe_pre[cid]['mar_l']) // 20 - 1) * 20
xmax = max(((ind.max() + spe_pre[cid]['mar_r']) // 20 + 1) * 20, xmin + 200)

In [10]:
TRIALS = 8000
n = 1
b_t0 = [0., 600.]

# initialization
A, y, tlist, t0_t, t0_delta, cha, left_wave, right_wave = wff.initial_params(wave[::wff.nshannon], spe_pre[cid], Tau, Sigma, gmu, Thres['lucyddm'], p, is_t0=True, is_delta=False, n=n)
# assert len(np.unique(np.diff(tlist))) == 1
s_cha = np.cumsum(cha)
# moving average filter of size 2*n+1
cha = np.pad(s_cha[2*n+1:], (n+1, n), 'edge') - np.pad(s_cha[:-(2*n+1)], (n+1, n), 'edge')
cha += 1e-8 # for completeness of the random walk.
p_cha = cha / np.sum(cha)
mu_t = abs(y.sum() / gmu)

# Eq. (9) where the columns of A are taken to be unit-norm.
mus = np.sqrt(np.diag(np.matmul(A.T, A)))
assert np.std(mus) < 1e-4, 'mus must be equal'
mus = mus[0]
A = A / mus
p1 = mu_t * wff.convolve_exp_norm(tlist - t0_t, Tau, Sigma) / n + 1e-8
sig2w = spe_pre[cid]['std'] ** 2
sig2s = (gsigma * mus / gmu) ** 2

nu_star, T_star, c_star, es_history, NPE_evo = wff.metropolis_fsmp(y, A, sig2w, sig2s, mus, p1, p_cha, mu_t)

ilp_cha = np.log(cha.sum()) - np.log(cha)
guess = ilp_cha[es_history['loc'].astype(int)]
es_history['loc'] = np.interp(es_history['loc'], xp=np.arange(0.5, len(tlist)), fp=tlist)
ans = opti.fmin_l_bfgs_b(lambda x: -np.sum(wff.log_convolve_exp_norm(es_history['loc'] - x, Tau, Sigma)), x0=[t0_t], approx_grad=True, bounds=[b_t0], maxfun=500000)
t00 = ans[0].item() if ans[-1]['warnflag'] == 0 else t0_t
def fit():
    mu = mu_t
    b_mu = [max(1e-8, mu - 5 * np.sqrt(mu)), mu + 5 * np.sqrt(mu)]
    def agg_NPE(t0):
        log_f = wff.log_convolve_exp_norm(es_history['loc'] - t0, Tau, Sigma) + guess
        return wff.jit_agg_NPE(es_history['step'], log_f, TRIALS)

    def t_t0(t0):
        nonlocal mu
        NPE, f_agg = agg_NPE(t0)
        ans = opti.fmin_l_bfgs_b(lambda μ: μ - special.logsumexp(NPE * np.log(μ / mu) + f_agg), x0=[mu], approx_grad=True, bounds=[b_mu], maxfun=500000)
        mu = ans[0].item()
        return ans[1]

    ans = opti.fmin_l_bfgs_b(t_t0, x0=[t00], approx_grad=True, bounds=[b_t0], maxfun=500000)
    t0 = ans[0].item()
    return mu, t0
mu, t0 = fit()

j = 0
xmmse_most = np.zeros(len(tlist))
while np.all(xmmse_most <= 0):
    maxindex = nu_star.argsort()[::-1][j]
    zx = y - np.dot(A, mus * c_star[maxindex])
    Phi_s = wff.Phi(y, A, c_star[maxindex], mus, sig2s, sig2w)
    invPhi = np.linalg.inv(Phi_s)
    xmmse_most = mus * c_star[maxindex] + np.matmul(np.diagflat(sig2s * c_star[maxindex]), np.matmul(A.T, np.matmul(invPhi, zx)))
    j += 0
pet = np.repeat(tlist[xmmse_most > 0], c_star[maxindex][xmmse_most > 0])
cha = np.repeat(xmmse_most[xmmse_most > 0] / mus / c_star[maxindex][xmmse_most > 0], c_star[maxindex][xmmse_most > 0])

pet, pwe = wff.clip(pet, cha, 0.0)
pwe = pwe
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_axes((.125, .12, .775, .77))
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.yaxis.get_major_formatter().set_powerlimits((0, 1))
ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
align.yaxes(ax, 0, ax2, 0)
wave_ylim = ax.get_ylim()
fig.savefig('Note/figures/fsmp.pgf')
fig.savefig('Note/figures/fsmp.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p)
print((t0 - t0truth[i]['T0']).item())

PEnum is 5
truth HitPosInWindow = [469.15547745 471.42601966 487.39357274 499.49537376 527.69280766], Weight = [0.60348736 1.25444866 1.37512431 1.0331502  1.1784267 ]
truth RSS = 1003.0466121337955
HitPosInWindow = [470. 472. 487. 499. 528.], Weight = [1.14345388 0.72399396 1.35121218 1.03809636 1.15761863]
wdist = 0.7088666011265952, cdiff = -4.841861596009473
RSS = 32.501677133723476
2.057835292425466


In [11]:
# fig = plt.figure(figsize=(8, 6))
# # fig.tight_layout()
# ax = fig.add_axes((.125, .12, .775, .77))
# ax.plot(wave, label='Waveform')
# ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
# ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
# ax.set_xlim(0, len(wave))
# wave_ylim = ax.get_ylim()
# ax.set_ylim(wave_ylim[0] * 1.05, wave_ylim[1] * 1.05)
# ax.legend()
# fig.savefig('Note/figures/wave.pgf')
# fig.savefig('Note/figures/wave.pdf')
# fig.savefig('Note/figures/wave.png')
# fig.clf()
# plt.close(fig)

fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_axes((.125, .12, .775, .77))
ax2 = ax.twinx()
ax2.vlines(truth['HitPosInWindow'], 0, truth['Charge'] / gmu, color='r', label='Charge', linewidth=1.0)
ax2.set_ylabel(r'$\mathrm{Charge}$')
ax2.set_ylim(wave_ylim[0] / 30, wave_ylim[1] / 30)
ax.plot(wave, label='Waveform')
# ax.set_xlim(xmin, xmax)
ax.set_xlim(0, len(wave) / 2)
ax.set_ylim(wave_ylim[0] * 0.7, wave_ylim[1] * 0.7)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
align.yaxes(ax, 0, ax2, 0)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.yaxis.get_major_formatter().set_powerlimits((0, 1))
ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax2.legend(lines + lines2, labels + labels2)
fig.savefig('Note/figures/wave.pgf')
fig.savefig('Note/figures/wave.pdf')
fig.savefig('Note/figures/wave.png')
fig.clf()
plt.close(fig)

fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
# gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
# ax = fig.add_subplot(gs[0, 0])
ax = fig.add_axes((.125, .12, .775, .77))
t = np.arange(0, 100, 0.1)
ax.plot(t, wff.spe(t, p[0], p[1], p[2]), color='b', label='Single PE response')
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.grid()
ax.set_xlim(0, 80)
ax.set_ylim(wave_ylim[0] * 0.7, wave_ylim[1] * 0.7)
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax.legend()
fig.savefig('Note/figures/spe.pgf')
fig.savefig('Note/figures/spe.pdf')
fig.savefig('Note/figures/spe.png')
plt.close()

# fig = plt.figure(figsize=(8, 6))
# # fig.tight_layout()
# ax = fig.add_axes((.125, .12, .775, .77))
# ax.vlines(truth['HitPosInWindow'], 0, truth['Charge'] / gmu, color='r', label='Charge')
# ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
# ax.set_ylabel(r'$\mathrm{Charge}$')
# ax.set_xlim(0, len(wave))
# ax.set_ylim(wave_ylim[0] / 20, wave_ylim[1] / 20)
# ax.axhline(y=0, color='k', linestyle='dashed', alpha=0.5)
# ax.legend()
# fig.savefig('Note/figures/charge.pgf')
# fig.savefig('Note/figures/charge.pdf')
# fig.savefig('Note/figures/charge.png')
# fig.clf()
# plt.close(fig)

In [12]:
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
# gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
# ax = fig.add_subplot(gs[0, 0])
ax = fig.add_axes((.125, .12, .775, .77))
ax2 = ax.twinx()
ax2.vlines(truth['HitPosInWindow'], 0, truth['Charge'] / gmu, color='r', label='Charge')
ax.plot(wave, label='Waveform')
ax.hlines(2, 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.yaxis.get_major_formatter().set_powerlimits((0, 1))
ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
ax.set_ylim(bottom=-5)
ax2.set_ylim(bottom=-5 / gmu)
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/goal.pgf')
fig.savefig('Note/figures/goal.pdf')
fig.clf()
plt.close(fig)

In [13]:
print(wave.sum())
print(truth['Charge'][truth['Charge'] > 0].sum()) # made by noise

869.1974059198186
871.1250094058131


In [14]:
pet, pwe = wff.threshold(wave, spe_pre[cid])
pet, pwe = wff.clip(pet, pwe, Thres['threshold'])
output = np.zeros(window)
output[pet] = pwe
alpha = opti.fmin_l_bfgs_b(lambda alpha: wff.rss_alpha(alpha, output, wave, mnecpu), x0=[0.01], approx_grad=True, bounds=[[1e-20, np.inf]], maxfun=50000)[0]
pwe = pwe * alpha
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_axes((.125, .12, .775, .77))
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
ax2.annotate('', xy=(pet.mean(), pwe.max()*1.1), xytext=(pet.mean()+pet.ptp(), pwe.max()*1.1), arrowprops=dict(facecolor='k', shrink=0.01, width=2, headwidth=4))
ax2.set_ylim(top=pwe.max()*1.2)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.yaxis.get_major_formatter().set_powerlimits((0, 1))
ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/threshold.pgf')
fig.savefig('Note/figures/threshold.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p, fold='/tmp')
t0 = wff.likelihoodt0(pet, char=pwe * gmu, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

PEnum is 5
truth HitPosInWindow = [469.15547745 471.42601966 487.39357274 499.49537376 527.69280766], Weight = [0.60348736 1.25444866 1.37512431 1.0331502  1.1784267 ]
truth RSS = 1003.0466121337955
HitPosInWindow = [466 467 468 469 470 471 472 473 474 475 476 477 478 479 480 481 482 483
 484 485 486 487 488 489 490 491 492 493 494 495 496 497 498 499 500 501
 502 503 504 505 506 507 508 509 524 525 526 527 528 529 530 531 532 533
 534 535 536 537], Weight = [0.04679002 0.07651529 0.1283519  0.14145558 0.17077878 0.18005191
 0.16295083 0.16173164 0.15600036 0.12022324 0.10785514 0.10705866
 0.08617904 0.05892233 0.05244189 0.04656959 0.0434028  0.05907324
 0.10041409 0.11774788 0.14615691 0.15260554 0.14441187 0.13267934
 0.1224666  0.11054901 0.10211239 0.08193174 0.085278   0.07084848
 0.10831113 0.10878063 0.13193294 0.1227105  0.12430346 0.11861385
 0.09976289 0.09164472 0.07213223 0.06184636 0.05062419 0.03496695
 0.04369119 0.04474155 0.05719246 0.07298535 0.10612047 0.10320779
 

In [15]:
pet, pwe = wff.findpeak(wave, spe_pre[cid])
pet, pwe = wff.clip(pet, pwe, Thres['findpeak'])
output = np.zeros(window)
output[pet] = pwe
alpha = opti.fmin_l_bfgs_b(lambda alpha: wff.rss_alpha(alpha, output, wave, mnecpu), x0=[0.01], approx_grad=True, bounds=[[1e-20, np.inf]], maxfun=50000)[0]
pwe = pwe * alpha
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_axes((.125, .12, .775, .77))
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=1.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
loc = pet + spe_pre[cid]['peak_c']
loc = loc[loc < window]
amp = wave[loc]
for j in range(len(loc)):
    ax.annotate('', xy=(loc[j], amp[j]+5), xytext=(loc[j], amp[j]+15), arrowprops=dict(facecolor='k', shrink=0.01, width=0.5, headwidth=2))
ax2.annotate('', xy=(pet.mean(), pwe.max()*1.1), xytext=(pet.mean()+pet.ptp(), pwe.max()*1.1), arrowprops=dict(facecolor='k', shrink=0.01, width=2, headwidth=4))
ax2.set_ylim(top=pwe.max()*1.2)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.yaxis.get_major_formatter().set_powerlimits((0, 1))
ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/findpeak.pgf')
fig.savefig('Note/figures/findpeak.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p, fold='/tmp')
t0 = wff.likelihoodt0(pet, char=pwe * gmu, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

PEnum is 5
truth HitPosInWindow = [469.15547745 471.42601966 487.39357274 499.49537376 527.69280766], Weight = [0.60348736 1.25444866 1.37512431 1.0331502  1.1784267 ]
truth RSS = 1003.0466121337955
HitPosInWindow = [472 489 500 529], Weight = [1.60835669 1.30957107 1.22689949 1.13247991]
wdist = 2.35038617939221, cdiff = -26.77229140635238
RSS = 739.9283743731916
4.852010259042913


In [16]:
pet, pwe = wff.waveformfft(wave, spe_pre[cid])
pet, pwe = wff.clip(pet, pwe, Thres['fftrans'])
output = np.zeros(window)
output[pet] = pwe
alpha = opti.fmin_l_bfgs_b(lambda alpha: wff.rss_alpha(alpha, output, wave, mnecpu), x0=[0.01], approx_grad=True, bounds=[[1e-20, np.inf]], maxfun=50000)[0]
pwe = pwe * alpha
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_axes((.125, .12, .775, .77))
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.yaxis.get_major_formatter().set_powerlimits((0, 1))
ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/fftrans.pgf')
fig.savefig('Note/figures/fftrans.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p, fold='/tmp')
t0 = wff.likelihoodt0(pet, char=pwe * gmu, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

PEnum is 5
truth HitPosInWindow = [469.15547745 471.42601966 487.39357274 499.49537376 527.69280766], Weight = [0.60348736 1.25444866 1.37512431 1.0331502  1.1784267 ]
truth RSS = 1003.0466121337955
HitPosInWindow = [467 468 469 470 471 472 473 474 475 484 485 486 487 488 489 490 497 498
 499 500 501 502 525 526 527 528 529 530 531], Weight = [0.13490864 0.20056036 0.25330008 0.28513933 0.29213436 0.27456125
 0.23611504 0.18261099 0.12073645 0.11227218 0.18484422 0.24092033
 0.26828681 0.2602826  0.21809638 0.15112532 0.1047817  0.16286432
 0.19923032 0.20521867 0.18049444 0.13249467 0.12004205 0.16491299
 0.19736295 0.20927639 0.19646852 0.1603138  0.10782198]
wdist = 1.8696699565298651, cdiff = 18.00603457141207
RSS = 153.70175366873292
2.614635617851434


In [17]:
pet, pwe = wff.lucyddm(wave, spe_pre[cid]['spe'])
pet, pwe = wff.clip(pet, pwe, Thres['lucyddm'])
output = np.zeros(window)
output[pet] = pwe
alpha = opti.fmin_l_bfgs_b(lambda alpha: wff.rss_alpha(alpha, output, wave, mnecpu), x0=[0.01], approx_grad=True, bounds=[[1e-20, np.inf]], maxfun=50000)[0]
pwe = pwe * alpha
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_axes((.125, .12, .775, .77))
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.yaxis.get_major_formatter().set_powerlimits((0, 1))
ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/lucyddm.pgf')
fig.savefig('Note/figures/lucyddm.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p, fold='/tmp')
t0 = wff.likelihoodt0(pet, char=pwe * gmu, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

PEnum is 5
truth HitPosInWindow = [469.15547745 471.42601966 487.39357274 499.49537376 527.69280766], Weight = [0.60348736 1.25444866 1.37512431 1.0331502  1.1784267 ]
truth RSS = 1003.0466121337955
HitPosInWindow = [469 471 472 487 488 499 500 527 528], Weight = [0.45424007 1.17786671 0.21697766 0.57416642 0.85227729 0.50602988
 0.48306009 0.45369149 0.7238494 ]
wdist = 0.604852940125768, cdiff = -0.39650806898040825
RSS = 10.01973168114615
2.774549745000627


In [18]:
with h5py.File('result/takara/char/' + file + '.h5', 'r', libver='latest', swmr=True) as ipt:
    photoelec = ipt['photoelectron'][:]
s = photoelec[(photoelec['TriggerNo'] == eid) & (photoelec['ChannelID'] == cid)]
pet = s['HitPosInWindow']
pwe = s['Charge']
pwe = pwe / gmu
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_axes((.125, .12, .775, .77))
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
# ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.yaxis.get_major_formatter().set_powerlimits((0, 1))
ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/takara.pgf')
fig.savefig('Note/figures/takara.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p, fold='/tmp')
t0 = wff.likelihoodt0(pet, char=pwe * gmu, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

PEnum is 5
truth HitPosInWindow = [469.15547745 471.42601966 487.39357274 499.49537376 527.69280766], Weight = [0.60348736 1.25444866 1.37512431 1.0331502  1.1784267 ]
truth RSS = 1003.0466121337955
HitPosInWindow = [470 471 472 487 488 499 500 528], Weight = [0.8683057  0.25852837 0.75238555 0.73799834 0.6444972  0.49203569
 0.50425877 1.15523982]
wdist = 0.8129154294613066, cdiff = -5.021948382194073
RSS = 19.534707730708483
2.9598339189554395


In [19]:
with h5py.File('result/mcmc/char/' + file + '.h5', 'r', libver='latest', swmr=True) as ipt:
    photoelec = ipt['photoelectron'][:]
s = photoelec[(photoelec['TriggerNo'] == eid) & (photoelec['ChannelID'] == cid)]
pet = s['HitPosInWindow']
pwe = s['Charge']
pwe = pwe / gmu
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_axes((.125, .12, .775, .77))
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.yaxis.get_major_formatter().set_powerlimits((0, 1))
ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/mcmc.pgf')
fig.savefig('Note/figures/mcmc.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p, fold='/tmp')
t0 = wff.likelihoodt0(pet, char=pwe * gmu, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

PEnum is 5
truth HitPosInWindow = [469.15547745 471.42601966 487.39357274 499.49537376 527.69280766], Weight = [0.60348736 1.25444866 1.37512431 1.0331502  1.1784267 ]
truth RSS = 1003.0466121337955
HitPosInWindow = [465.  467.  468.5 469.5 471.5 472.  472.5 473.  474.  475.5 476.  482.5
 483.5 486.  486.5 487.  487.5 488.5 489.  489.5 490.5 491.  492.  494.5
 495.  496.  497.  498.5 500.  525.  527.  527.5 528.  529.  529.5 530.
 530.5 531.  531.5 532. ], Weight = [0.00004056 0.00000161 0.50646792 0.00000606 1.3935919  0.00002333
 0.00002546 0.0000265  0.0000705  0.00002203 0.00000904 0.00001444
 0.00000447 0.00000808 0.00000127 0.000059   1.37724389 0.00001398
 0.00000858 0.00001346 0.00002844 0.00002556 0.00001106 0.00000025
 0.0000299  0.00003845 0.0000416  0.36676836 0.65237934 0.00000705
 0.00002421 1.15792123 0.00000763 0.00003973 0.00002968 0.0000268
 0.00001788 0.00000645 0.00002019 0.00004749]
wdist = 0.6243421832926701, cdiff = 1.6777548751523437
RSS = 8.834620517524197
2.67

In [20]:
# pet, pwe = wff.xiaopeip(wave, spe_pre[cid], eta=0)
pet, pwe = wff.xiaopeip(wave, spe_pre[cid], Tau, Sigma, Thres['lucyddm'], p, eta=0)
pet, pwe = wff.clip(pet, pwe, Thres['xiaopeip'])
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_axes((.125, .12, .775, .77))
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
# ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.yaxis.get_major_formatter().set_powerlimits((0, 1))
ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/xiaopeip.pgf')
fig.savefig('Note/figures/xiaopeip.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p, fold='/tmp')
t0 = wff.likelihoodt0(pet, char=pwe * gmu, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

PEnum is 5
truth HitPosInWindow = [469.15547745 471.42601966 487.39357274 499.49537376 527.69280766], Weight = [0.60348736 1.25444866 1.37512431 1.0331502  1.1784267 ]
truth RSS = 1003.0466121337955
HitPosInWindow = [467. 468. 470. 471. 472. 487. 488. 499. 500. 527. 528.], Weight = [0.08310591 0.00032206 0.97774912 0.01575962 0.81853973 0.59466171
 0.79408098 0.493306   0.51071231 0.32664292 0.83637863]
wdist = 0.9068197801619707, cdiff = 1.0594602473590373
RSS = 6.49430638214507
2.7278145296716048


In [21]:
methods = ['lucyddm', 'xiaopeip', 'takara', 'fsmp', 'mcmc']

for m in methods:
    with h5py.File('result/' + m + '/dist/' + file + '.h5', 'r', libver='latest', swmr=True) as distfile:
        dt = distfile['Record'][:]
    N = np.percentile(dt['wdist'], 95)
    M = 500

    penum = np.unique(dt['NPE'])
    l = min(50, penum.max())
    wdist_stats = np.full((l, 6), np.nan)
    edist_stats = np.full((l, 6), np.nan)
    for i in range(l):
        vali = dt['NPE'] == i+1
        if np.sum(vali) == 0:
            continue
        dtwpi = dt['wdist'][vali]
        dtepi = dt['RSS'][vali]
        wdist_stats[i, 0] = np.median(dtwpi)
        wdist_stats[i, 1] = np.median(np.abs(dtwpi - np.median(dtwpi)))
        wdist_stats[i, 2] = np.mean(dtwpi)
        wdist_stats[i, 3] = np.std(dtwpi)
        wdist_stats[i, 4] = np.percentile(dtwpi, norm.cdf(-1) * 1e2)
        wdist_stats[i, 5] = np.percentile(dtwpi, norm.cdf(1) * 1e2)
        edist_stats[i, 0] = np.median(dtepi)
        edist_stats[i, 1] = np.median(np.abs(dtepi - np.median(dtepi)))
        edist_stats[i, 2] = np.mean(dtepi)
        edist_stats[i, 3] = np.std(dtepi)
        edist_stats[i, 4] = np.percentile(dtepi, norm.cdf(-1) * 1e2)
        edist_stats[i, 5] = np.percentile(dtepi, norm.cdf(1) * 1e2)

    L = len(dt)
    data = dt['wdist']
    fig = plt.figure(figsize=(8, 6))
    ax1 = fig.add_axes((.125, .12, .6, .77))
    # boxdict = ax1.boxplot(np.array([dt['wdist'][dt['NPE'] == i+1] for i in range(l)], dtype=np.object), sym='', patch_artist=True)
    ax1.plot(np.arange(1, l + 1), wdist_stats[:, 0], color='b')
    
    ax1.errorbar(np.arange(1, l + 1), 
                wdist_stats[:, 0], 
                yerr=[wdist_stats[:, 0] - wdist_stats[:, 4], wdist_stats[:, 5] - wdist_stats[:, 0]], 
                fmt='o', elinewidth=1, capsize=3, color='b', label=r'$D_w$')
    
    ax1.fill_between(np.arange(1, l + 1), 
                    wdist_stats[:, 4], 
                    wdist_stats[:, 5], fc='k', alpha=0.1, color=None)

    ax1.set_xticks(np.arange(1, 16, 2))
    ax1.set_xticklabels(np.arange(1, 16, 2).astype(str))
    ax1.set_xlim(0, l + 1)
    # ax1.set_ylim(0, max([boxdict['whiskers'][2 * i + 1].get_xydata()[1, 1] for i in range(l)]) * 1.05)
    # ax1.set_ylim(0, wdist_stats[:, 5].max() * 1.05)
    ax1.set_xlabel(r'$N_{\mathrm{PE}}$')
    ax1.set_ylabel(r'$\mathrm{Wasserstein\ Distance}/\si{ns}$')
    ax1.legend()

    ax2 = fig.add_axes((.725, .12, .175, .77))
    bins = np.linspace(0, data[data < np.percentile(data, 90)].std() * 3 + np.mean(data), 41)
    ax2.hist(data, bins=bins, density=1, orientation='horizontal')

    ax2.set_xlabel(r'$\mathrm{arb.\ unit}$')
    ax2.set_xlim(0, ax2.get_xlim()[1] * 1.05)
    ax2.set_xticks([])
    ax2.set_yticks([])
    ax2.set_ylim(0, bins[-1])
    ax1.set_ylim(0, bins[-1])
    fig.savefig('Note/figures/' + m + 'chargestats.pgf')
    fig.savefig('Note/figures/' + m + 'chargestats.pdf')
    plt.close(fig)

In [22]:
t = np.arange(0, 1000, 0.1) / gmu

pdf = np.zeros_like(t)
tlist = np.arange(-50, 200)

for mu in tqdm(Mu * wff.convolve_exp_norm(tlist, Tau, Sigma)):
    for i in range(1, 15):
        pdf += mu * poisson.pmf(i, mu) * norm.pdf(t, loc=1, scale=gsigma / gmu / np.sqrt(i))

100%|██████████| 250/250 [00:01<00:00, 146.96it/s]


In [23]:
methods = ['lucyddm', 'xiaopeip', 'takara', 'fsmp']
colors = {'truth':'k', 'lucyddm':'y', 'xiaopeip':'c', 'takara':'C0', 'fsmp':'r'}
fig = plt.figure(figsize=(10, 6))
fig.tight_layout()
ax = fig.add_axes((.1, .12, .85, .80))
t = np.arange(0, 1000, 0.1) / gmu
# ax.plot(t, norm.pdf(t, loc=1, scale=gsigma / gmu) / (1 - norm.cdf(0, loc=1, scale=gsigma / gmu)), color=colors['truth'], alpha=0.2)
ax.plot(t, pdf / pdf.sum() / np.diff(t)[0], label='$\mathrm{ChargePDF}$', color=colors['truth'])
# th = 160 * 5 * 1e-4
th = 10 / gmu
labels = {'truth':'\mathrm{Truth}', 'lucyddm':'\mathrm{LucyDDM}', 'xiaopeip':'\mathrm{Fitting}', 'takara':'\mathrm{CNN}', 'fsmp':'\mathrm{FSMP}', 'fsmpwave':'\mathrm{FSMP}'}
for m in methods:
    ch = h5py.File('result/' + m + '/char/' + file + '.h5', 'r', libver='latest', swmr=True)
    cha = ch['photoelectron']['Charge'] / gmu
    ax.hist(cha[cha > th], bins=np.linspace(th, 400 / gmu, 101), label='$'+labels[m]+'$', histtype='step', density=True, color=colors[m], linewidth=2.)
ax.set_xlim(10 / gmu, 310 / gmu)
ax.set_yticks([])
# ax.yaxis.get_major_formatter().set_powerlimits((0, 1))
# ax.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax.legend()
# ax.set_xlabel('$\mathrm{Charge}/\si{mV\cdot ns}$')
ax.set_xlabel('$\mathrm{Charge}$')
ax.set_ylabel(r'$\mathrm{Normalized\ Count}$')
plt.savefig('Note/figures/recchargehist.png')
plt.savefig('Note/figures/recchargehist.pdf')
plt.savefig('Note/figures/recchargehist.pgf')
plt.show()

In [24]:
t = np.arange(0, 1000, 0.1) / gmu

pdf = np.zeros_like(t)
b = 0.5
tlist = np.arange(-50, 200, b)

for mu in tqdm(25 * wff.convolve_exp_norm(tlist, Tau, Sigma) * b):
    for i in range(1, 15):
        pdf += mu * poisson.pmf(i, mu) * norm.pdf(t, loc=1, scale=gsigma / gmu / np.sqrt(i))

100%|██████████| 500/500 [00:03<00:00, 151.00it/s]


In [25]:
methods = ['lucyddm', 'xiaopeip', 'takara', 'fsmp']
colors = {'truth':'k', 'lucyddm':'y', 'xiaopeip':'c', 'takara':'C0', 'fsmp':'r'}
fig = plt.figure(figsize=(10, 6))
fig.tight_layout()
ax = fig.add_axes((.1, .12, .85, .80))
ax.plot(t, norm.pdf(t, loc=1, scale=gsigma / gmu) / (1 - norm.cdf(0, loc=1, scale=gsigma / gmu)), color=colors['truth'], alpha=0.2)
ax.plot(t, pdf / pdf.sum() / np.diff(t)[0], label='$\mathrm{ChargePDF}$', color=colors['truth'])
# th = 160 * 5 * 1e-4
th = 10 / gmu
labels = {'truth':'\mathrm{Truth}', 'lucyddm':'\mathrm{LucyDDM}', 'xiaopeip':'\mathrm{Fitting}', 'takara':'\mathrm{CNN}', 'fsmp':'\mathrm{FSMP}', 'fsmpwave':'\mathrm{FSMP}'}
for m in methods:
    ch = h5py.File('result/' + m + '/char/15.0-20-5' + '.h5', 'r', libver='latest', swmr=True)
    cha = ch['photoelectron']['Charge'] / gmu
    ax.hist(cha[cha > th], bins=np.linspace(th, 400 / gmu, 101), label='$'+labels[m]+'$', histtype='step', density=True, color=colors[m], linewidth=2.)
ax.set_xlim(10 / gmu, 310 / gmu)
ax.set_yticks([])
# ax.yaxis.get_major_formatter().set_powerlimits((0, 1))
# ax.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'))
ax.legend()
# ax.set_xlabel('$\mathrm{Charge}/\si{mV\cdot ns}$')
ax.set_xlabel('$\mathrm{Charge}$')
ax.set_ylabel(r'$\mathrm{Normalized\ Count}$')
plt.savefig('Note/figures/recchargehist25.png')
# plt.savefig('Note/figures/recchargehist.pdf')
# plt.savefig('Note/figures/recchargehist.pgf')
plt.show()